In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#DC GAN
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

# Define hyperparameters (same as used during training)
image_size = 64
image_channels = 3

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for preprocessing the input image
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define the Discriminator class
class Discriminator(nn.Module):
    def __init__(self, image_channels, hidden_dim=64):
        super(Discriminator, self).__init__()

        self.main = nn.Sequential(
            # Input: image_channels x 64 x 64
            nn.Conv2d(image_channels, hidden_dim, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            # Input: hidden_dim x 32 x 32
            nn.Conv2d(hidden_dim, hidden_dim * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(hidden_dim * 2),
            nn.LeakyReLU(0.2, inplace=True),

            # Input: hidden_dim * 2 x 16 x 16
            nn.Conv2d(hidden_dim * 2, hidden_dim * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(hidden_dim * 4),
            nn.LeakyReLU(0.2, inplace=True),

            # Input: hidden_dim * 4 x 8 x 8
            nn.Conv2d(hidden_dim * 4, hidden_dim * 8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(hidden_dim * 8),
            nn.LeakyReLU(0.2, inplace=True),

            # Input: hidden_dim * 8 x 4 x 4
            nn.Conv2d(hidden_dim * 8, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()  # Output: 1 x 1 x 1
        )

    def forward(self, input):
      return self.main(input)

# Load the pre-trained discriminator model on the CPU
discriminator = Discriminator(image_channels).to(device)
discriminator.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/GAN_gate/GATE/dc_models/netD_epoch_430.pth", map_location=device))
discriminator.eval()  # Set the discriminator to evaluation mode

# Define the classify_image function
def classify_image(image_path, threshold=0.8):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    # Use the discriminator to classify the image
    with torch.no_grad():
        output = discriminator(image)

    # Apply a sigmoid activation function to get the probabilities
    probabilities = torch.sigmoid(output)

    # Compute the mean probability over the batch
    mean_probability = torch.mean(probabilities).item()

    # Check if the image is fake or genuine based on the mean probability and threshold
    if mean_probability > threshold:
        return "1"
    else:
        return "0"

import os

# Directory containing the image files
image_dir = "/content/drive/MyDrive/Colab Notebooks/GAN_gate/GATE/Fake_images/Retouching_images"  # Replace with the path to your image directory

# Threshold for classification
threshold = 0.503

# List all image files in the directory
image_files = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if filename.endswith((".jpg", ".png", ".jpeg"))]

# Initialize counters for '0's and '1's
total_count_0 = 0
total_count_1 = 0

# Define a function to classify an image and return counts
def classify_and_count(image_path):
    result = classify_image(image_path, threshold=threshold)

    # Count the number of '0's and '1's in the classification result
    count_0 = result.count('0')
    count_1 = result.count('1')

    # Print the individual counts for each image
    #print(f"The image at {image_path} is classified as: {result}")
    print(result)
    #print(f"Count 0: {count_0}, Count 1: {count_1}")

    return count_0, count_1

# Import the multiprocessing module with Pool
import multiprocessing

if __name__ == "__main__":
    # Create a multiprocessing pool with the number of desired processes
    num_processes = multiprocessing.cpu_count()  # You can adjust this as needed
    pool = multiprocessing.Pool(processes=num_processes)

    # Use the pool to classify images in parallel and collect the results
    results = pool.map(classify_and_count, image_files)

    # Close the pool to release resources
    pool.close()
    pool.join()

    # Sum up the counts from all images
    total_count_0 = sum(count_0 for count_0, _ in results)
    total_count_1 = sum(count_1 for _, count_1 in results)

    # Display the total counts
    print(f"Total Count 0: {total_count_0}, Total Count 1: {total_count_1}")
    print("Th:",threshold)

# Test an input image for different thresholds
#image_path_to_test = "/content/drive/MyDrive/GATE/Generated_dc/epoch_348_image_001.png"  # Replace with the path of the image you want to test
#thresholds = [0.5 + i * 0.01 for i in range(0, 20)]

#for threshold in thresholds:
    #result = classify_image(image_path_to_test, threshold=threshold)
    #print(f"The image is classified as: {result} (Threshold: {threshold})")


10

1
0
0
0
0
1
1
0
0
0
0
0
00

0
0
10

11

01

01

0
0
0
1
00

1
0
0
0
0
0
1
0
01

0
0
11

1
1
1
0
0
0
1
11

0
1
0
1
1
01

1
1
1
1
0
0
0
00

0
1
01

1
1
1
1
0
1
1
1
1
1
1
0
1
0
0
1
1
0
1
1
1
1
1
1
0
Total Count 0: 50, Total Count 1: 50
Th: 0.503


In [ ]:
#W gan
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

# Define hyperparameters (same as used during training)
image_size = 64
image_channels = 3

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for preprocessing the input image
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define the critic (discriminator) network
class Discriminator(nn.Module):

    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()

        self.disc = nn.Sequential(
            #size = 3*64*64
            nn.Conv2d(channels_img, features_d, kernel_size = 4, stride = 2, padding = 1), # Size : 32*32
            nn.LeakyReLU(0.2),

            nn.Conv2d(features_d, features_d*2, kernel_size = 4, stride = 2, padding = 1), # size = 16*16
            nn.BatchNorm2d(features_d*2),
            nn.LeakyReLU(0.2),

            nn.Conv2d(features_d*2, features_d*4, kernel_size = 4, stride = 2, padding = 1), # size = 8*8
            nn.BatchNorm2d(features_d*4),
            nn.LeakyReLU(0.2),

            nn.Conv2d(features_d*4, features_d*8, kernel_size = 4, stride = 2, padding = 1), # size = 4*4
            nn.BatchNorm2d(features_d*8),
            nn.LeakyReLU(0.2),

            nn.Conv2d(features_d*8, 1, kernel_size = 4, stride = 2, padding = 0) #1*1

        )


    def forward(self, x):
        return self.disc(x)
# Load the pre-trained discriminator model on the CPU
# You need to specify the number of features for the discriminator
features_d = 64  # You can adjust this value as needed

# Create an instance of the Discriminator class
discriminator = Discriminator(image_channels, features_d).to(device)
#discriminator = Discriminator(image_channels).to(device)
discriminator.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/GAN_gate/GATE/wg_models/discriminator_epoch750.pth", map_location=device))
discriminator.eval()  # Set the discriminator to evaluation mode

# Define the classify_image function
def classify_image(image_path, threshold=0.8):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    # Use the discriminator to classify the image
    with torch.no_grad():
        output = discriminator(image)

    # Apply a sigmoid activation function to get the probabilities
    probabilities = torch.sigmoid(output)

    # Compute the mean probability over the batch
    mean_probability = torch.mean(probabilities).item()

    # Check if the image is fake or genuine based on the mean probability and threshold
    if mean_probability > threshold:
        return "1"
    else:
        return "0"
import os

# Directory containing the image files
image_dir = "/content/drive/MyDrive/Colab Notebooks/GAN_gate/GATE/Fake_images/Retouching_images"  # Replace with the path to your image directory

# Threshold for classification
threshold = 0.3

# List all image files in the directory
image_files = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if filename.endswith((".jpg", ".png", ".jpeg"))]

# Initialize counters for '0's and '1's
total_count_0 = 0
total_count_1 = 0

# Define a function to classify an image and return counts
def classify_and_count(image_path):
    result = classify_image(image_path, threshold=threshold)

    # Count the number of '0's and '1's in the classification result
    count_0 = result.count('0')
    count_1 = result.count('1')

    # Print the individual counts for each image
    #print(f"The image at {image_path} is classified as: {result}")
    #print(f"Count 0: {count_0}, Count 1: {count_1}")
    print(result)
    return count_0, count_1

# Import the multiprocessing module with Pool
import multiprocessing

if __name__ == "__main__":
    # Create a multiprocessing pool with the number of desired processes
    num_processes = multiprocessing.cpu_count()  # You can adjust this as needed
    pool = multiprocessing.Pool(processes=num_processes)

    # Use the pool to classify images in parallel and collect the results
    results = pool.map(classify_and_count, image_files)

    # Close the pool to release resources
    pool.close()
    pool.join()

    # Sum up the counts from all images
    total_count_0 = sum(count_0 for count_0, _ in results)
    total_count_1 = sum(count_1 for _, count_1 in results)

    # Display the total counts
    print(f"Total Count 0: {total_count_0}, Total Count 1: {total_count_1}")
    print("thr:",threshold)





# Test an input image for different thresholds
#image_path_to_test = "/content/drive/MyDrive/TEST/Copy of O_0001.jpg"  # Replace with the path of the image you want to test
#thresholds = [0.5 + i * 0.01 for i in range(0, 20)]

#for threshold in thresholds:
    #result = classify_image(image_path_to_test, threshold=threshold)
    #print(f"The image is classified as: {result} (Threshold: {threshold})")


01

1
0
1
0
1
1
0
1
1
0
1
1
10

10

10

00

00

0
1
00

1
1
1
0
10

1
0
0
1
10

1
0
11

11

0
1
10

11

01

01

01

0
0
1
1
1
1
11

1
1
1
1
0
10

00

0
1
11

01

11

10

01

1
1
01

10

00

10

1
1
1
Total Count 0: 41, Total Count 1: 59
thr: 0.3
